In [14]:
# Import the modules
import cv2
import joblib
from skimage.feature import hog
from sklearn import datasets
import numpy as np

In [3]:
# Load the classifier
clf,pp = joblib.load("digits_cls1.pkl")

In [4]:
# Read the input image 
im = cv2.imread("photo_1.jpg")

In [5]:
# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)

In [6]:
# Threshold the image
ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)

# Find contours in the image
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Print the number of contours found
print(f"Number of contours found: {len(ctrs)}")

Number of contours found: 8


In [7]:
# Get rectangles contains each contour
rects = [cv2.boundingRect(ctr) for ctr in ctrs]

In [8]:
for rect in rects:
    # Draw the rectangles
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
    # Make the rectangular region around the digit
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
    # Resize the image
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))
    # Calculate the HOG features
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
    roi_hog_fd = pp.transform(np.array([roi_hog_fd], 'float64'))
    # Predict digit
    nbr = clf.predict(roi_hog_fd)
    cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)


In [9]:
cv2.namedWindow("Resulting Image with Rectangular ROIs", cv2.WINDOW_NORMAL)
cv2.imshow("Resulting Image with Rectangular ROIs", im)
cv2.waitKey()

-1

In [10]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [11]:
with open("predictions.txt", "w") as f:
    for rect in rects:
        # Draw the rectangles
        cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
        # Make the rectangular region around the digit
        leng = int(rect[3] * 1.6)
        pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
        pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
        roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
        # Resize the image
        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
        roi = cv2.dilate(roi, (3, 3))
        # Calculate the HOG features
        roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
        roi_hog_fd = pp.transform(np.array([roi_hog_fd], 'float64'))
        # Predict digit
        nbr = clf.predict(roi_hog_fd)
        # Display the predicted digit on the image
        cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
        # Print the predicted digit to the output (e.g., terminal)
        print(f"Predicted digit at position ({rect[0]}, {rect[1]}): {int(nbr[0])}")
        # Write the predicted digit to the file
        f.write(f"Predicted digit at position ({rect[0]}, {rect[1]}): {int(nbr[0])}\n")

# Display the result
cv2.imshow("Resulting Image with Rectangles", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicted digit at position (248, 268): 2
Predicted digit at position (148, 266): 5
Predicted digit at position (63, 262): 0
Predicted digit at position (425, 261): 1
Predicted digit at position (324, 260): 2
Predicted digit at position (559, 257): 5
Predicted digit at position (645, 256): 9
Predicted digit at position (474, 256): 8


In [1]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

# Load the dataset
dataset = fetch_openml('mnist_784')
features = np.array(dataset.data, dtype='float64')
labels = np.array(dataset.target, dtype='int')

# Extract the HOG features for the entire dataset
list_hog_ft = []
for feature in features:
    ft = hog(feature.reshape((28, 28)), orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
    list_hog_ft.append(ft)
hog_features = np.array(list_hog_ft, dtype='float64')

# Fit the StandardScaler with the HOG features
scaler = StandardScaler()
hog_features = scaler.fit_transform(hog_features)

# Fit the LinearSVC with the scaled HOG features
clf = LinearSVC()
clf.fit(hog_features, labels)

# Assume im, im_th, and rects are already defined
# Example:
# im = cv2.imread('image.png')
# im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
# ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)
# rects = [cv2.boundingRect(ctr) for ctr in ctrs]

# Initialize a list to store predicted digits
predicted_digits = []

# Open a file to write the results
with open("predictions.txt", "w") as f:
    for rect in rects:
        # Draw the rectangles
        cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
        # Make the rectangular region around the digit
        leng = int(rect[3] * 1.6)
        pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
        pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
        roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
        # Resize the image
        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
        roi = cv2.dilate(roi, (3, 3))
        # Calculate the HOG features
        roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
        roi_hog_fd = scaler.transform(np.array([roi_hog_fd], 'float64'))
        # Predict digit
        nbr = clf.predict(roi_hog_fd)
        # Display the predicted digit on the image
        cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
        # Print the predicted digit to the output (e.g., terminal) and write to file
        result = f"Predicted digit at position ({rect[0]}, {rect[1]}): {int(nbr[0])}"
        print(result)
        f.write(result + "\n")
        # Append predicted digit to the list
        predicted_digits.append(str(int(nbr[0])))

# Combine the list of digits into a single string
result_string = ' '.join(predicted_digits)
print("Combined result:", result_string)

# Display the result
cv2.imshow("Resulting Image with Rectangles", im)
cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'rects' is not defined

In [13]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

# Gunakan tipe data yang lebih kecil
array = np.zeros((1024,), dtype=np.int32)
# Load the dataset
dataset = fetch_openml('mnist_784')
features = np.array(dataset.data, dtype='float64')
labels = np.array(dataset.target, dtype='int')

# Extract the HOG features for the entire dataset
list_hog_ft = []
for feature in features:
    ft = hog(feature.reshape((28, 28)), orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
    list_hog_ft.append(ft)
hog_features = np.array(list_hog_ft, dtype='float64')

# Fit the StandardScaler with the HOG features
scaler = StandardScaler()
hog_features = scaler.fit_transform(hog_features)

# Fit the LinearSVC with the scaled HOG features
clf = LinearSVC()
clf.fit(hog_features, labels)

# Assume im, im_th, and rects are already defined
# Example:
# im = cv2.imread('image.png')
# im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
# ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)
# rects = [cv2.boundingRect(ctr) for ctr in ctrs]

# Initialize a list to store predicted digits and their positions
predicted_digits = []

# Open a file to write the results
with open("predictions.txt", "w") as f:
    for rect in rects:
        # Make the rectangular region around the digit
        leng = int(rect[3] * 1.6)
        pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
        pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
        roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
        # Resize the image
        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
        roi = cv2.dilate(roi, (3, 3))
        # Calculate the HOG features
        roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
        roi_hog_fd = scaler.transform(np.array([roi_hog_fd], 'float64'))
        # Predict digit
        nbr = clf.predict(roi_hog_fd)
        # Display the predicted digit on the image
        cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
        # Print the predicted digit to the output (e.g., terminal) and write to file
        result = f"Predicted digit at position ({rect[0]}, {rect[1]}): {int(nbr[0])}"
        print(result)
        f.write(result + "\n")
        # Append predicted digit and its position to the list
        predicted_digits.append((rect[0], str(int(nbr[0]))))

# Sort digits by their x-coordinate (left to right)
predicted_digits.sort(key=lambda x: x[0])

# Combine the sorted list of digits into a single string
result_string = ' '.join(digit for _, digit in predicted_digits)
print("Combined result:", result_string)

# Display the result
cv2.imshow("Resulting Image with Rectangles", im)
cv2.waitKey(0)
cv2.destroyAllWindows()


c:\Users\Khaula sayyidatunadi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:72: RuntimeWarning: Invalid cache, redownloading file
  warn("Invalid cache, redownloading file", RuntimeWarning)


MemoryError: Unable to allocate 8.00 KiB for an array with shape (1024,) and data type int64